# Multiclass Classifier

Abbiamo visto come utilizzare un classifier per distinguere le immagini che contenevano un '5' dalle immagini che 'non contenevano un 5'. Supponiamo ora di voler essere in grado di distinguere tutti i numeri da 0 a 9, allora ci serve un multiclass classifier.

Il SVM è un algoritmo che è nativamente binario, cioè non è in grado di gestire più classi contemporaneamete. Tuttavia esistono delle strategie che permettono di usare classificatori binari anche nel contesto delle multiclassi.

### One - Versus - Rest  

Un modo per poter classificare i numeri da 0 a 9 è quello di fare il training di 10 modelli di SVM diversi:
* '1' contro 'non 1'
* '2' contro 'non 2'
* ...
* '9' contro 'non 9'

Una volta che si hanno i 10 classificatori, all'arrivo di una nuova immagine si calcola per ognuno il **decision-score** (nel caso del SVM la decision_function), la classe con lo score più alto vince!

In [ ]:
from sklearn import datasets
import numpy as np

mnist = datasets.load_digits()
mnist['target'] = mnist['target'].astype(np.int)

In [ ]:
from sklearn.model_selection import train_test_split

X = mnist['data']
y = mnist['target']

X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.2, shuffle=True, random_state = 6)

In [ ]:
from sklearn.svm import SVC

#decision_function_shape: decision_function_shape{‘ovo’, ‘ovr’}, default=’ovr’

svc_clf = SVC(decision_function_shape = 'ovr')
svc_clf.fit(X_train, y_train)

In [ ]:
svc_clf.predict([X_train[0]])

In [ ]:
y_train[0]

In [ ]:
svc_clf.decision_function([X_train[0]])

In [ ]:
svc_clf.classes_

Lo score massimo si ha per la classe 8

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

def show_image(x):
    plt.imshow(x.reshape(8,8), cmap="binary")
    plt.axis("off")
    plt.show()

In [ ]:
show_image(X_train[0])

In [ ]:
import scikitplot as skplt

y_pred = svc_clf.predict(X_train)
skplt.metrics.plot_confusion_matrix(y_train, y_pred)



In [ ]:
from sklearn.metrics import precision_score, recall_score
precision_score = precision_score(y_train, y_pred,average = 'weighted')
recall_score = recall_score(y_train, y_pred, average = 'weighted')
print('Precision Score: {} Recall Score {}'.format(precision_score,recall_score))

ESERCIZIO: 
1. usando la funzione make_blobs(usa random_state=6) genera dei dati associati a 3 classi diverse
2. fai il training del modello SVC con kernel lineare;
3. genera il grafico dei punti e dei support_vectors;
4. fai predizioni sulla classe relativa al punto (-3,1) e controlla i valori della decision function;

[soluzione](./soluzione/oneVsRest.ipynb)

## One - Versus - One

Un'altra strategia è quella di fare training per ogni coppia di classi: una per distinguere 0 e 1, una per distinguere 0 e 2,...
Se ci sono N classi ci sarà bisogno di fare il training di N(N-1)/2

#### Per il caso del MNIST dobbiamo fare training di 45 classificatori!

Se si vuole fare predizioni su una nuova immagine bisognerà far fare predizioni a tutti i 45 classificatori, 
la classe che vince più duelli sarà quella vincente!

La scelta se applicare un approccio OvR o OvO dipende dal particolare algoritmo che state usando:

|         |OvR               |OvO          | 
|----------------------|-------------|-----------|
| Numero di classificatori | N          | N(N-1)/2       |
| Dataset di training      |tutto il dataset  | solo i dati relativi alle due classi |

In [ ]:
svc_clf_ovo = SVC(decision_function_shape = 'ovo')
svc_clf_ovo.fit(X_train, y_train)

In [ ]:
dec_function = svc_clf_ovo.decision_function([X_train[0]])
dec_function

In [ ]:
dec_function.shape

In [ ]:
svc_clf_ovo.predict([X_train[0]])

In [ ]:
import scikitplot as skplt

y_pred = svc_clf_ovo.predict(X_train)
skplt.metrics.plot_confusion_matrix(y_train, y_pred, normalize=True)

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision_score = precision_score(y_train, y_pred,average = 'weighted')
recall_score = recall_score(y_train, y_pred, average = 'weighted')
print('Precision Score: {} Recall Score {}'.format(precision_score,recall_score))